In [680]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
from datetime import datetime
from itertools import combinations, permutations
from dateutil.relativedelta import relativedelta

In [681]:
df = pd.read_excel('PORTFOLIO.xlsx')

In [682]:
df_gpd = df.groupby('ticker').agg({'data_ini': 'min', 'data_fin': 'max', 'setor':'last'})

In [683]:
tickers = df.ticker.unique()

In [684]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [672]:
# Download data
df_list = []
errors = []

utc_from = df.data_ini.min() - np.timedelta64(700, 'D')
utc_to = df.data_fin.max()

for i, row in df_gpd.iterrows():
    try: 
        rates = mt5.copy_rates_range(row.name, mt5.TIMEFRAME_D1,utc_from, utc_to)

        # criamos a partir dos dados obtidos DataFrame
        rates_frame = pd.DataFrame(rates)
        # convertemos o tempo em segundos no formato datetime
        rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
        rates_frame['ticker'] = row.name

        df_list.append(rates_frame[['time', 'close', 'ticker']])
    except Exception:
        errors.append(row.name)

In [673]:
errors

['BIDI11', 'GNDI3', 'HGTX3', 'LCAM3']

In [674]:
df_prices = pd.concat(df_list).pivot(columns='ticker',  index='time', values='close')

In [675]:
df_prices

ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2017-07-31,16.89,11.33,12.82,NaN,27.29,5.66,22.51,14.27,14.33,18.54,...,9.31,9.55,31.94,4.42,23.43,NaN,5.83,26.03,6.81,18.07
2017-08-01,16.97,11.41,12.88,NaN,27.19,5.78,22.98,14.34,14.51,18.58,...,9.37,9.53,32.29,4.52,23.25,NaN,5.67,25.52,6.88,17.97
2017-08-02,17.11,11.56,13.20,NaN,27.41,5.83,23.91,14.49,14.71,18.79,...,9.63,9.60,30.71,4.70,23.29,NaN,5.58,25.83,7.01,17.27
2017-08-03,16.99,11.11,13.10,NaN,28.03,5.84,24.06,14.23,14.70,18.75,...,9.57,9.99,31.09,4.72,22.97,NaN,5.67,25.86,6.89,16.34
2017-08-04,16.99,11.23,13.10,NaN,27.79,5.82,24.21,14.33,14.72,18.31,...,9.55,10.34,30.82,4.71,23.38,NaN,5.67,26.03,6.88,16.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-27,13.72,19.80,14.03,14.70,13.31,11.47,33.13,15.06,18.15,24.92,...,12.60,24.26,12.54,9.25,78.05,16.70,2.20,46.46,25.80,13.96
2022-06-28,13.52,19.69,13.42,14.62,13.00,11.15,33.25,14.93,17.89,25.45,...,12.56,23.78,12.71,9.12,79.45,16.96,2.08,46.24,26.02,13.58
2022-06-29,13.61,19.54,13.44,14.38,12.69,10.95,33.08,14.71,17.58,25.48,...,12.50,23.27,12.56,9.01,78.79,16.90,2.09,45.42,25.90,13.40


In [676]:
# df_prices.to_excel('database.xlsx')

In [479]:
df.data_ini.unique()

array(['2019-07-01T00:00:00.000000000', '2020-01-01T00:00:00.000000000',
       '2020-07-01T00:00:00.000000000', '2021-01-01T00:00:00.000000000',
       '2021-07-01T00:00:00.000000000', '2022-01-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [581]:

# Permutation over same sectors stocks

# Initiates a dictionary with key as initial date 
permut = {data_ini:[data_fin, []] for data_ini, data_fin in zip(df.data_ini.unique(), df.data_fin.unique())}

# Loop over initial dates
for data in df.data_ini.unique():   
    
    # Select stocks with same initial date
    df_sem = df.loc[df.data_ini==data]
    
    # Loops over sectors to create a permutation
    for setor in df_sem.setor.unique():
        
        # Select stocks of same sector and appends its permutations on dictionary
        tickers = df_sem.loc[df_sem.setor == setor, 'ticker'].to_list()
        permut[data][-1].extend(list(permutations(tickers, 2)))

In [582]:
permut

{numpy.datetime64('2019-07-01T00:00:00.000000000'): [numpy.datetime64('2019-12-31T00:00:00.000000000'),
  [('AZUL4', 'GOLL4'),
   ('GOLL4', 'AZUL4'),
   ('ABEV3', 'BRFS3'),
   ('ABEV3', 'JBSS3'),
   ('ABEV3', 'MRFG3'),
   ('BRFS3', 'ABEV3'),
   ('BRFS3', 'JBSS3'),
   ('BRFS3', 'MRFG3'),
   ('JBSS3', 'ABEV3'),
   ('JBSS3', 'BRFS3'),
   ('JBSS3', 'MRFG3'),
   ('MRFG3', 'ABEV3'),
   ('MRFG3', 'BRFS3'),
   ('MRFG3', 'JBSS3'),
   ('CCRO3', 'ECOR3'),
   ('CCRO3', 'EMBR3'),
   ('CCRO3', 'RAIL3'),
   ('CCRO3', 'WEGE3'),
   ('ECOR3', 'CCRO3'),
   ('ECOR3', 'EMBR3'),
   ('ECOR3', 'RAIL3'),
   ('ECOR3', 'WEGE3'),
   ('EMBR3', 'CCRO3'),
   ('EMBR3', 'ECOR3'),
   ('EMBR3', 'RAIL3'),
   ('EMBR3', 'WEGE3'),
   ('RAIL3', 'CCRO3'),
   ('RAIL3', 'ECOR3'),
   ('RAIL3', 'EMBR3'),
   ('RAIL3', 'WEGE3'),
   ('WEGE3', 'CCRO3'),
   ('WEGE3', 'ECOR3'),
   ('WEGE3', 'EMBR3'),
   ('WEGE3', 'RAIL3'),
   ('CYRE3', 'MRVE3'),
   ('MRVE3', 'CYRE3'),
   ('COGN3', 'YDUQ3'),
   ('YDUQ3', 'COGN3'),
   ('BRML3', 'IGTI3'),

In [651]:
len(df_prices.loc[data_ini: data_fin])

126

In [652]:
df_prices.loc[data_ini: data_fin].shape

(126, 89)

In [655]:
permut

{numpy.datetime64('2019-07-01T00:00:00.000000000'): [numpy.datetime64('2019-12-31T00:00:00.000000000'),
  [('AZUL4', 'GOLL4'),
   ('GOLL4', 'AZUL4'),
   ('ABEV3', 'BRFS3'),
   ('ABEV3', 'JBSS3'),
   ('ABEV3', 'MRFG3'),
   ('BRFS3', 'ABEV3'),
   ('BRFS3', 'JBSS3'),
   ('BRFS3', 'MRFG3'),
   ('JBSS3', 'ABEV3'),
   ('JBSS3', 'BRFS3'),
   ('JBSS3', 'MRFG3'),
   ('MRFG3', 'ABEV3'),
   ('MRFG3', 'BRFS3'),
   ('MRFG3', 'JBSS3'),
   ('CCRO3', 'ECOR3'),
   ('CCRO3', 'EMBR3'),
   ('CCRO3', 'RAIL3'),
   ('CCRO3', 'WEGE3'),
   ('ECOR3', 'CCRO3'),
   ('ECOR3', 'EMBR3'),
   ('ECOR3', 'RAIL3'),
   ('ECOR3', 'WEGE3'),
   ('EMBR3', 'CCRO3'),
   ('EMBR3', 'ECOR3'),
   ('EMBR3', 'RAIL3'),
   ('EMBR3', 'WEGE3'),
   ('RAIL3', 'CCRO3'),
   ('RAIL3', 'ECOR3'),
   ('RAIL3', 'EMBR3'),
   ('RAIL3', 'WEGE3'),
   ('WEGE3', 'CCRO3'),
   ('WEGE3', 'ECOR3'),
   ('WEGE3', 'EMBR3'),
   ('WEGE3', 'RAIL3'),
   ('CYRE3', 'MRVE3'),
   ('MRVE3', 'CYRE3'),
   ('COGN3', 'YDUQ3'),
   ('YDUQ3', 'COGN3'),
   ('BRML3', 'IGTI3'),

In [ ]:
from backtest import Executer

In [664]:
df_trading.shape[0] - (train_size-1)

124

In [662]:
days_of_trading

124

In [658]:
# Loops over dict with initial, final dates and permutations
# And filters the dataframe with prices with 365 days (one year)
# of formation data and 6 months of trading data
count = 0
for key, value in permut.items():
    
    data_ini = key
    data_pre =  data_ini - np.timedelta64(365, 'D')
    data_fin = value[0]
    
    perm = value[1]
    
    df_trading = df_prices.loc[data_pre: data_fin]
    
    train_size = df_trading[data_pre:data_ini].shape[0] + 1        
    days_of_trading = df_prices.loc[data_ini: data_fin].shape[0]
    
    print(days_of_trading)
    for i in range(days_of_tradingrading):

        d = df_trading.iloc[: train_size + i][-train_size:]
        display(d)
    count+=1
    if count > 1:
        break

126


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2018-07-02,16.40,9.52,25.86,NaN,20.70,5.86,23.77,12.93,14.32,17.54,...,11.59,8.68,20.34,6.21,37.19,14.16,5.84,29.95,7.40,22.21
2018-07-03,16.35,9.50,26.69,NaN,20.72,5.93,23.98,13.25,14.98,17.70,...,11.54,8.79,20.33,6.66,36.58,14.27,5.74,29.99,7.56,22.79
2018-07-04,16.36,9.63,27.94,NaN,20.99,6.00,24.66,13.53,15.19,17.69,...,11.68,8.84,20.25,7.06,36.63,14.23,5.72,30.19,7.52,23.10
2018-07-05,16.63,9.72,27.31,NaN,20.42,5.97,24.09,13.44,15.16,17.63,...,11.63,8.93,20.23,6.95,37.90,14.03,5.58,29.95,7.55,22.88
2018-07-06,16.35,9.80,27.30,NaN,20.79,6.18,23.82,13.54,15.20,17.69,...,11.74,8.89,20.63,6.91,38.39,14.52,5.48,29.04,7.72,23.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-26,16.66,20.05,31.60,NaN,42.39,10.71,45.32,22.04,25.20,25.94,...,10.61,14.06,18.90,7.49,40.34,21.81,4.98,35.33,10.15,27.11
2019-06-27,16.62,20.30,31.45,NaN,42.47,10.92,45.41,22.11,25.08,25.90,...,10.52,14.13,18.58,7.56,40.44,21.81,4.97,35.24,10.25,27.82
2019-06-28,16.47,20.17,31.53,NaN,43.29,10.88,45.53,22.07,24.98,25.86,...,10.55,14.25,18.22,7.79,40.53,21.81,5.07,34.64,10.25,27.81


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2018-07-03,16.35,9.50,26.69,NaN,20.72,5.93,23.98,13.25,14.98,17.70,...,11.54,8.79,20.33,6.66,36.58,14.27,5.74,29.99,7.56,22.79
2018-07-04,16.36,9.63,27.94,NaN,20.99,6.00,24.66,13.53,15.19,17.69,...,11.68,8.84,20.25,7.06,36.63,14.23,5.72,30.19,7.52,23.10
2018-07-05,16.63,9.72,27.31,NaN,20.42,5.97,24.09,13.44,15.16,17.63,...,11.63,8.93,20.23,6.95,37.90,14.03,5.58,29.95,7.55,22.88
2018-07-06,16.35,9.80,27.30,NaN,20.79,6.18,23.82,13.54,15.20,17.69,...,11.74,8.89,20.63,6.91,38.39,14.52,5.48,29.04,7.72,23.67
2018-07-10,16.44,9.82,26.78,NaN,20.68,6.35,23.62,13.33,15.00,17.83,...,11.66,9.06,20.15,6.77,37.77,15.09,5.35,29.88,8.04,23.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-27,16.62,20.30,31.45,NaN,42.47,10.92,45.41,22.11,25.08,25.90,...,10.52,14.13,18.58,7.56,40.44,21.81,4.97,35.24,10.25,27.82
2019-06-28,16.47,20.17,31.53,NaN,43.29,10.88,45.53,22.07,24.98,25.86,...,10.55,14.25,18.22,7.79,40.53,21.81,5.07,34.64,10.25,27.81
2019-07-01,16.40,20.38,30.84,NaN,42.09,11.08,45.48,22.20,25.01,26.20,...,10.39,14.16,17.90,7.74,41.96,21.37,5.12,34.48,10.37,27.18


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2018-07-04,16.36,9.63,27.94,NaN,20.99,6.00,24.66,13.53,15.19,17.69,...,11.68,8.84,20.25,7.06,36.63,14.23,5.72,30.19,7.52,23.10
2018-07-05,16.63,9.72,27.31,NaN,20.42,5.97,24.09,13.44,15.16,17.63,...,11.63,8.93,20.23,6.95,37.90,14.03,5.58,29.95,7.55,22.88
2018-07-06,16.35,9.80,27.30,NaN,20.79,6.18,23.82,13.54,15.20,17.69,...,11.74,8.89,20.63,6.91,38.39,14.52,5.48,29.04,7.72,23.67
2018-07-10,16.44,9.82,26.78,NaN,20.68,6.35,23.62,13.33,15.00,17.83,...,11.66,9.06,20.15,6.77,37.77,15.09,5.35,29.88,8.04,23.38
2018-07-11,16.19,9.84,25.75,NaN,20.89,6.35,24.09,13.29,14.94,17.77,...,11.63,9.00,20.11,6.47,37.26,15.17,5.65,29.74,7.95,23.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-28,16.47,20.17,31.53,NaN,43.29,10.88,45.53,22.07,24.98,25.86,...,10.55,14.25,18.22,7.79,40.53,21.81,5.07,34.64,10.25,27.81
2019-07-01,16.40,20.38,30.84,NaN,42.09,11.08,45.48,22.20,25.01,26.20,...,10.39,14.16,17.90,7.74,41.96,21.37,5.12,34.48,10.37,27.18
2019-07-02,16.69,20.85,30.68,NaN,42.41,10.90,44.90,22.18,25.09,26.04,...,10.37,14.65,17.63,7.65,40.20,20.59,5.45,35.41,10.36,27.67


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2018-07-05,16.63,9.72,27.31,NaN,20.42,5.97,24.09,13.44,15.16,17.63,...,11.63,8.93,20.23,6.95,37.90,14.03,5.58,29.95,7.55,22.88
2018-07-06,16.35,9.80,27.30,NaN,20.79,6.18,23.82,13.54,15.20,17.69,...,11.74,8.89,20.63,6.91,38.39,14.52,5.48,29.04,7.72,23.67
2018-07-10,16.44,9.82,26.78,NaN,20.68,6.35,23.62,13.33,15.00,17.83,...,11.66,9.06,20.15,6.77,37.77,15.09,5.35,29.88,8.04,23.38
2018-07-11,16.19,9.84,25.75,NaN,20.89,6.35,24.09,13.29,14.94,17.77,...,11.63,9.00,20.11,6.47,37.26,15.17,5.65,29.74,7.95,23.36
2018-07-12,16.49,9.90,27.03,NaN,21.72,6.55,24.55,13.60,15.38,17.97,...,11.87,9.02,20.04,7.07,38.47,15.62,5.59,29.66,8.06,23.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-01,16.40,20.38,30.84,NaN,42.09,11.08,45.48,22.20,25.01,26.20,...,10.39,14.16,17.90,7.74,41.96,21.37,5.12,34.48,10.37,27.18
2019-07-02,16.69,20.85,30.68,NaN,42.41,10.90,44.90,22.18,25.09,26.04,...,10.37,14.65,17.63,7.65,40.20,20.59,5.45,35.41,10.36,27.67
2019-07-03,17.37,20.85,31.13,NaN,44.16,11.15,45.83,22.60,25.43,26.18,...,10.68,14.83,17.67,7.91,40.12,20.94,5.93,35.64,10.43,27.82


...

ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2019-06-26,16.66,20.05,31.60,NaN,42.39,10.71,45.32,22.04,25.20,25.94,...,10.61,14.06,18.90,7.49,40.34,21.81,4.98,35.33,10.15,27.11
2019-06-27,16.62,20.30,31.45,NaN,42.47,10.92,45.41,22.11,25.08,25.90,...,10.52,14.13,18.58,7.56,40.44,21.81,4.97,35.24,10.25,27.82
2019-06-28,16.47,20.17,31.53,NaN,43.29,10.88,45.53,22.07,24.98,25.86,...,10.55,14.25,18.22,7.79,40.53,21.81,5.07,34.64,10.25,27.81
2019-07-01,16.40,20.38,30.84,NaN,42.09,11.08,45.48,22.20,25.01,26.20,...,10.39,14.16,17.90,7.74,41.96,21.37,5.12,34.48,10.37,27.18
2019-07-02,16.69,20.85,30.68,NaN,42.41,10.90,44.90,22.18,25.09,26.04,...,10.37,14.65,17.63,7.65,40.20,20.59,5.45,35.41,10.36,27.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-19,13.03,29.48,101.39,NaN,21.80,15.30,29.98,15.33,17.22,25.78,...,13.50,22.27,17.33,6.15,44.18,19.53,15.18,44.15,22.45,34.43
2020-06-22,13.28,29.03,99.83,NaN,20.63,15.10,29.36,15.06,16.62,25.82,...,13.43,22.32,17.03,6.13,44.04,19.15,14.90,43.14,22.15,34.39
2020-06-23,13.11,29.98,100.70,NaN,22.50,15.68,29.62,15.07,16.60,25.74,...,13.24,22.52,17.14,6.76,44.51,18.98,15.03,42.67,22.40,36.19


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2019-06-27,16.62,20.30,31.45,NaN,42.47,10.92,45.41,22.11,25.08,25.90,...,10.52,14.13,18.58,7.56,40.44,21.81,4.97,35.24,10.25,27.82
2019-06-28,16.47,20.17,31.53,NaN,43.29,10.88,45.53,22.07,24.98,25.86,...,10.55,14.25,18.22,7.79,40.53,21.81,5.07,34.64,10.25,27.81
2019-07-01,16.40,20.38,30.84,NaN,42.09,11.08,45.48,22.20,25.01,26.20,...,10.39,14.16,17.90,7.74,41.96,21.37,5.12,34.48,10.37,27.18
2019-07-02,16.69,20.85,30.68,NaN,42.41,10.90,44.90,22.18,25.09,26.04,...,10.37,14.65,17.63,7.65,40.20,20.59,5.45,35.41,10.36,27.67
2019-07-03,17.37,20.85,31.13,NaN,44.16,11.15,45.83,22.60,25.43,26.18,...,10.68,14.83,17.67,7.91,40.12,20.94,5.93,35.64,10.43,27.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-22,13.28,29.03,99.83,NaN,20.63,15.10,29.36,15.06,16.62,25.82,...,13.43,22.32,17.03,6.13,44.04,19.15,14.90,43.14,22.15,34.39
2020-06-23,13.11,29.98,100.70,NaN,22.50,15.68,29.62,15.07,16.60,25.74,...,13.24,22.52,17.14,6.76,44.51,18.98,15.03,42.67,22.40,36.19
2020-06-24,12.88,29.38,103.60,NaN,21.10,15.79,28.46,14.74,16.06,25.40,...,13.05,22.10,17.17,6.46,44.35,18.89,14.58,42.15,22.37,34.32


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2019-06-28,16.47,20.17,31.53,NaN,43.29,10.88,45.53,22.07,24.98,25.86,...,10.55,14.25,18.22,7.79,40.53,21.81,5.07,34.64,10.25,27.81
2019-07-01,16.40,20.38,30.84,NaN,42.09,11.08,45.48,22.20,25.01,26.20,...,10.39,14.16,17.90,7.74,41.96,21.37,5.12,34.48,10.37,27.18
2019-07-02,16.69,20.85,30.68,NaN,42.41,10.90,44.90,22.18,25.09,26.04,...,10.37,14.65,17.63,7.65,40.20,20.59,5.45,35.41,10.36,27.67
2019-07-03,17.37,20.85,31.13,NaN,44.16,11.15,45.83,22.60,25.43,26.18,...,10.68,14.83,17.67,7.91,40.12,20.94,5.93,35.64,10.43,27.82
2019-07-04,17.49,21.04,33.03,NaN,46.75,11.54,46.58,23.10,25.98,26.43,...,11.05,15.13,17.81,8.25,40.41,21.38,6.24,36.49,10.67,28.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-23,13.11,29.98,100.70,NaN,22.50,15.68,29.62,15.07,16.60,25.74,...,13.24,22.52,17.14,6.76,44.51,18.98,15.03,42.67,22.40,36.19
2020-06-24,12.88,29.38,103.60,NaN,21.10,15.79,28.46,14.74,16.06,25.40,...,13.05,22.10,17.17,6.46,44.35,18.89,14.58,42.15,22.37,34.32
2020-06-25,13.35,29.81,102.61,NaN,21.20,16.50,29.16,15.06,16.44,25.97,...,13.24,22.75,17.00,6.42,44.84,19.42,14.55,42.95,23.91,33.95


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2019-07-01,16.40,20.38,30.84,NaN,42.09,11.08,45.48,22.20,25.01,26.20,...,10.39,14.16,17.90,7.74,41.96,21.37,5.12,34.48,10.37,27.18
2019-07-02,16.69,20.85,30.68,NaN,42.41,10.90,44.90,22.18,25.09,26.04,...,10.37,14.65,17.63,7.65,40.20,20.59,5.45,35.41,10.36,27.67
2019-07-03,17.37,20.85,31.13,NaN,44.16,11.15,45.83,22.60,25.43,26.18,...,10.68,14.83,17.67,7.91,40.12,20.94,5.93,35.64,10.43,27.82
2019-07-04,17.49,21.04,33.03,NaN,46.75,11.54,46.58,23.10,25.98,26.43,...,11.05,15.13,17.81,8.25,40.41,21.38,6.24,36.49,10.67,28.70
2019-07-05,17.49,21.20,34.48,NaN,48.45,11.76,46.43,23.28,26.21,27.04,...,11.02,15.89,17.77,8.45,39.39,21.59,6.35,36.18,10.77,29.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-24,12.88,29.38,103.60,NaN,21.10,15.79,28.46,14.74,16.06,25.40,...,13.05,22.10,17.17,6.46,44.35,18.89,14.58,42.15,22.37,34.32
2020-06-25,13.35,29.81,102.61,NaN,21.20,16.50,29.16,15.06,16.44,25.97,...,13.24,22.75,17.00,6.42,44.84,19.42,14.55,42.95,23.91,33.95
2020-06-26,13.13,28.64,100.51,NaN,20.00,16.03,28.11,14.63,15.94,24.94,...,13.08,21.95,16.38,6.25,44.54,18.65,14.29,42.17,24.29,32.33


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2019-07-02,16.69,20.85,30.68,NaN,42.41,10.90,44.90,22.18,25.09,26.04,...,10.37,14.65,17.63,7.65,40.20,20.59,5.45,35.41,10.36,27.67
2019-07-03,17.37,20.85,31.13,NaN,44.16,11.15,45.83,22.60,25.43,26.18,...,10.68,14.83,17.67,7.91,40.12,20.94,5.93,35.64,10.43,27.82
2019-07-04,17.49,21.04,33.03,NaN,46.75,11.54,46.58,23.10,25.98,26.43,...,11.05,15.13,17.81,8.25,40.41,21.38,6.24,36.49,10.67,28.70
2019-07-05,17.49,21.20,34.48,NaN,48.45,11.76,46.43,23.28,26.21,27.04,...,11.02,15.89,17.77,8.45,39.39,21.59,6.35,36.18,10.77,29.00
2019-07-08,17.44,20.75,35.14,NaN,48.73,11.73,46.38,23.43,26.09,26.77,...,10.88,15.78,17.85,8.42,39.85,22.20,6.75,36.74,10.83,30.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-25,13.35,29.81,102.61,NaN,21.20,16.50,29.16,15.06,16.44,25.97,...,13.24,22.75,17.00,6.42,44.84,19.42,14.55,42.95,23.91,33.95
2020-06-26,13.13,28.64,100.51,NaN,20.00,16.03,28.11,14.63,15.94,24.94,...,13.08,21.95,16.38,6.25,44.54,18.65,14.29,42.17,24.29,32.33
2020-06-29,13.42,29.08,103.48,NaN,20.73,16.49,29.54,15.00,16.49,25.30,...,12.84,22.11,16.72,6.30,44.54,18.87,15.38,41.75,24.26,33.44


In [607]:
index_ini.item()

248

In [483]:
#index number of starting date
df_prices.reset_index().loc[df_prices.reset_index().time >= df.data_ini.unique()[3]].index[0]

619

In [583]:
for key, value in permut.items():
    print(key, len(value[-1]))

2019-07-01T00:00:00.000000000 298
2020-01-01T00:00:00.000000000 412
2020-07-01T00:00:00.000000000 478
2021-01-01T00:00:00.000000000 494
2021-07-01T00:00:00.000000000 538
2022-01-01T00:00:00.000000000 658


In [584]:
dates[0]

numpy.datetime64('2019-07-01T00:00:00.000000000')

In [500]:
dates = list(permut.keys())
d = 5
ini = dates[d] - np.timedelta64(365*2, 'D')
fin = dates[d]

In [514]:
df_prices

ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2018-07-02,16.40,9.52,25.86,NaN,20.70,5.86,23.77,12.93,14.32,17.54,...,11.59,8.68,20.34,6.21,37.19,14.16,5.84,29.95,7.40,22.21
2018-07-03,16.35,9.50,26.69,NaN,20.72,5.93,23.98,13.25,14.98,17.70,...,11.54,8.79,20.33,6.66,36.58,14.27,5.74,29.99,7.56,22.79
2018-07-04,16.36,9.63,27.94,NaN,20.99,6.00,24.66,13.53,15.19,17.69,...,11.68,8.84,20.25,7.06,36.63,14.23,5.72,30.19,7.52,23.10
2018-07-05,16.63,9.72,27.31,NaN,20.42,5.97,24.09,13.44,15.16,17.63,...,11.63,8.93,20.23,6.95,37.90,14.03,5.58,29.95,7.55,22.88
2018-07-06,16.35,9.80,27.30,NaN,20.79,6.18,23.82,13.54,15.20,17.69,...,11.74,8.89,20.63,6.91,38.39,14.52,5.48,29.04,7.72,23.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-27,13.72,19.80,14.03,14.70,13.31,11.47,33.13,15.06,18.15,24.92,...,12.60,24.26,12.54,9.25,78.05,16.70,2.20,46.46,25.80,13.96
2022-06-28,13.52,19.69,13.42,14.62,13.00,11.15,33.25,14.93,17.89,25.45,...,12.56,23.78,12.71,9.12,79.45,16.96,2.08,46.24,26.02,13.58
2022-06-29,13.61,19.54,13.44,14.38,12.69,10.95,33.08,14.71,17.58,25.48,...,12.50,23.27,12.56,9.01,78.79,16.90,2.09,45.42,25.90,13.40


In [515]:
dates

[numpy.datetime64('2019-07-01T00:00:00.000000000'),
 numpy.datetime64('2020-01-01T00:00:00.000000000'),
 numpy.datetime64('2020-07-01T00:00:00.000000000'),
 numpy.datetime64('2021-01-01T00:00:00.000000000'),
 numpy.datetime64('2021-07-01T00:00:00.000000000'),
 numpy.datetime64('2022-01-01T00:00:00.000000000')]

In [556]:
dates[0]-np.timedelta64(365, 'D')

numpy.datetime64('2018-07-01T00:00:00.000000000')

In [558]:
d = 0
df_prices.loc[dates[d]-np.timedelta64(500, 'D'):dates[d]]

ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2018-07-02,16.40,9.52,25.86,NaN,20.70,5.86,23.77,12.93,14.32,17.54,...,11.59,8.68,20.34,6.21,37.19,14.16,5.84,29.95,7.40,22.21
2018-07-03,16.35,9.50,26.69,NaN,20.72,5.93,23.98,13.25,14.98,17.70,...,11.54,8.79,20.33,6.66,36.58,14.27,5.74,29.99,7.56,22.79
2018-07-04,16.36,9.63,27.94,NaN,20.99,6.00,24.66,13.53,15.19,17.69,...,11.68,8.84,20.25,7.06,36.63,14.23,5.72,30.19,7.52,23.10
2018-07-05,16.63,9.72,27.31,NaN,20.42,5.97,24.09,13.44,15.16,17.63,...,11.63,8.93,20.23,6.95,37.90,14.03,5.58,29.95,7.55,22.88
2018-07-06,16.35,9.80,27.30,NaN,20.79,6.18,23.82,13.54,15.20,17.69,...,11.74,8.89,20.63,6.91,38.39,14.52,5.48,29.04,7.72,23.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-25,16.73,20.05,31.61,NaN,42.60,10.89,44.36,21.95,24.89,25.36,...,10.66,13.83,18.86,7.38,40.19,21.19,5.04,35.15,10.16,27.32
2019-06-26,16.66,20.05,31.60,NaN,42.39,10.71,45.32,22.04,25.20,25.94,...,10.61,14.06,18.90,7.49,40.34,21.81,4.98,35.33,10.15,27.11
2019-06-27,16.62,20.30,31.45,NaN,42.47,10.92,45.41,22.11,25.08,25.90,...,10.52,14.13,18.58,7.56,40.44,21.81,4.97,35.24,10.25,27.82


In [559]:
indexes = df_prices.index.get_indexer(target=dates, method='bfill').tolist()
# indexes.insert(0,0)
indexes

[244, 370, 493, 619, 741, 866]

In [574]:
for i in range(0, len(dates)-1):
    display(df_prices.loc[dates[i]-np.timedelta64(365, 'D'): dates[i+1]])

ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2018-07-02,16.40,9.52,25.86,NaN,20.70,5.86,23.77,12.93,14.32,17.54,...,11.59,8.68,20.34,6.21,37.19,14.16,5.84,29.95,7.40,22.21
2018-07-03,16.35,9.50,26.69,NaN,20.72,5.93,23.98,13.25,14.98,17.70,...,11.54,8.79,20.33,6.66,36.58,14.27,5.74,29.99,7.56,22.79
2018-07-04,16.36,9.63,27.94,NaN,20.99,6.00,24.66,13.53,15.19,17.69,...,11.68,8.84,20.25,7.06,36.63,14.23,5.72,30.19,7.52,23.10
2018-07-05,16.63,9.72,27.31,NaN,20.42,5.97,24.09,13.44,15.16,17.63,...,11.63,8.93,20.23,6.95,37.90,14.03,5.58,29.95,7.55,22.88
2018-07-06,16.35,9.80,27.30,NaN,20.79,6.18,23.82,13.54,15.20,17.69,...,11.74,8.89,20.63,6.91,38.39,14.52,5.48,29.04,7.72,23.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-20,17.80,31.97,61.12,NaN,57.00,14.39,44.57,23.10,24.84,30.93,...,14.18,22.38,22.57,8.14,42.91,25.75,11.37,40.49,16.38,46.14
2019-12-23,17.84,32.32,61.03,NaN,57.60,13.73,45.91,23.56,25.00,31.10,...,14.28,22.12,23.23,8.32,42.75,26.54,11.67,40.61,16.35,46.98
2019-12-26,18.09,32.47,61.15,NaN,57.07,13.67,46.43,23.81,25.31,30.95,...,14.46,21.97,23.48,8.32,42.91,26.49,11.80,41.35,16.86,46.60


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2019-01-02,14.93,13.84,41.43,NaN,36.33,8.00,40.14,19.62,22.22,22.08,...,10.90,9.28,24.30,8.39,39.96,20.63,4.38,31.99,8.57,23.39
2019-01-03,15.10,13.86,41.86,NaN,36.37,8.34,40.30,20.03,22.44,22.32,...,10.81,9.50,23.85,8.21,38.33,21.10,4.31,31.60,8.77,23.37
2019-01-04,15.30,13.76,40.47,NaN,36.25,8.04,40.30,19.59,22.27,22.11,...,10.87,9.76,23.69,8.69,40.82,21.18,4.27,31.36,8.87,23.73
2019-01-07,15.24,13.72,38.68,NaN,35.30,8.00,40.04,19.75,22.53,21.78,...,11.22,9.55,22.96,8.70,40.60,21.30,4.24,31.45,8.73,23.26
2019-01-08,14.93,13.99,40.47,NaN,34.72,8.02,39.58,19.97,22.67,21.46,...,11.65,9.16,22.58,8.45,40.99,20.59,4.04,30.76,8.66,23.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-25,13.35,29.81,102.61,NaN,21.20,16.50,29.16,15.06,16.44,25.97,...,13.24,22.75,17.00,6.42,44.84,19.42,14.55,42.95,23.91,33.95
2020-06-26,13.13,28.64,100.51,NaN,20.00,16.03,28.11,14.63,15.94,24.94,...,13.08,21.95,16.38,6.25,44.54,18.65,14.29,42.17,24.29,32.33
2020-06-29,13.42,29.08,103.48,NaN,20.73,16.49,29.54,15.00,16.49,25.30,...,12.84,22.11,16.72,6.30,44.54,18.87,15.38,41.75,24.26,33.44


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2019-07-02,16.69,20.85,30.68,NaN,42.41,10.90,44.90,22.18,25.09,26.04,...,10.37,14.65,17.63,7.65,40.20,20.59,5.45,35.41,10.36,27.67
2019-07-03,17.37,20.85,31.13,NaN,44.16,11.15,45.83,22.60,25.43,26.18,...,10.68,14.83,17.67,7.91,40.12,20.94,5.93,35.64,10.43,27.82
2019-07-04,17.49,21.04,33.03,NaN,46.75,11.54,46.58,23.10,25.98,26.43,...,11.05,15.13,17.81,8.25,40.41,21.38,6.24,36.49,10.67,28.70
2019-07-05,17.49,21.20,34.48,NaN,48.45,11.76,46.43,23.28,26.21,27.04,...,11.02,15.89,17.77,8.45,39.39,21.59,6.35,36.18,10.77,29.00
2019-07-08,17.44,20.75,35.14,NaN,48.73,11.73,46.38,23.43,26.09,26.77,...,10.88,15.78,17.85,8.42,39.85,22.20,6.75,36.74,10.83,30.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-22,14.92,40.08,74.31,NaN,35.27,18.27,34.23,18.74,20.97,26.92,...,13.75,27.41,21.68,12.30,71.27,20.13,15.94,40.14,35.49,31.46
2020-12-23,14.92,40.71,73.95,NaN,37.70,18.19,34.79,19.14,21.45,26.90,...,13.92,27.56,22.01,12.88,71.62,20.19,16.13,40.23,35.40,32.15
2020-12-28,15.17,41.44,75.36,NaN,37.77,18.64,35.18,19.31,21.66,27.45,...,14.04,27.88,22.19,12.73,71.58,20.24,16.59,40.59,36.80,32.32


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2020-01-02,18.07,32.20,63.54,NaN,58.80,13.32,46.96,24.57,26.11,31.22,...,14.63,22.67,23.48,8.46,43.50,27.01,11.73,40.58,17.05,44.83
2020-01-03,17.82,32.20,62.68,NaN,56.76,12.94,46.88,24.18,26.12,30.88,...,14.66,23.16,23.20,8.38,43.18,26.86,11.48,41.05,16.83,43.89
2020-01-06,17.90,31.69,62.29,NaN,55.00,12.81,46.26,24.04,25.65,31.39,...,14.64,22.63,22.70,8.22,42.93,25.51,11.48,40.65,16.87,44.73
2020-01-07,17.94,32.41,62.14,NaN,56.82,13.23,45.91,23.63,25.21,31.40,...,14.80,22.72,23.20,8.30,43.24,25.48,11.65,41.55,17.00,43.90
2020-01-08,17.85,32.77,62.97,NaN,56.92,13.19,45.49,23.27,24.82,31.39,...,14.95,22.42,23.02,8.19,43.25,25.35,11.60,41.97,16.36,43.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-25,16.39,48.98,66.62,16.81,45.97,15.62,30.37,19.38,22.85,22.46,...,11.40,36.64,18.04,16.69,98.06,26.99,15.72,39.82,32.88,33.75
2021-06-28,16.92,50.53,67.94,17.00,45.33,15.90,30.14,19.23,22.74,22.63,...,11.55,37.38,18.23,16.77,96.49,26.68,16.03,39.98,33.33,33.77
2021-06-29,16.72,50.08,67.38,17.05,44.53,15.95,29.94,19.07,22.60,22.29,...,11.33,37.18,17.61,16.97,98.15,25.74,16.02,39.35,34.01,33.14


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2020-07-01,13.52,28.94,109.21,NaN,20.67,16.85,29.21,14.79,16.30,24.80,...,13.49,23.42,17.46,6.46,43.67,19.16,15.99,41.88,25.56,33.62
2020-07-02,13.34,29.69,105.22,NaN,20.76,16.90,29.29,14.91,16.35,24.65,...,13.57,23.79,17.31,6.47,44.43,19.32,15.39,42.08,26.04,33.41
2020-07-03,13.31,29.00,106.12,NaN,21.15,16.99,29.59,14.91,16.46,24.86,...,14.22,23.86,17.11,6.57,44.25,19.51,15.55,42.77,26.25,33.68
2020-07-06,13.52,29.28,115.28,NaN,22.05,16.99,30.62,15.91,17.46,25.10,...,14.27,24.02,17.22,6.70,45.30,19.76,15.96,42.64,25.86,34.15
2020-07-07,13.60,29.35,114.16,NaN,21.50,17.19,29.40,15.26,16.74,24.89,...,13.92,24.18,16.75,6.56,45.14,19.69,15.86,42.39,26.20,33.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,15.55,36.40,30.06,13.98,26.46,11.08,27.84,14.54,17.24,19.84,...,12.71,28.63,13.74,14.10,76.22,20.93,4.50,46.95,32.99,20.18
2021-12-27,15.53,36.51,31.20,13.53,25.67,11.01,27.89,14.69,17.44,19.96,...,12.73,28.70,14.02,14.27,76.03,21.19,4.86,46.67,32.69,19.92
2021-12-28,15.52,36.61,30.35,12.99,25.75,10.87,27.73,14.68,17.37,20.00,...,12.60,28.21,14.07,13.99,73.96,21.35,4.95,46.39,32.81,20.70


In [527]:
for i in range(1,len(indexes)-1):
#     display(df_prices.iloc[:i]
    display(df_prices.iloc[indexes[i-2]: indexes[i]])

ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2018-07-02,16.40,9.52,25.86,NaN,20.70,5.86,23.77,12.93,14.32,17.54,...,11.59,8.68,20.34,6.21,37.19,14.16,5.84,29.95,7.40,22.21
2018-07-03,16.35,9.50,26.69,NaN,20.72,5.93,23.98,13.25,14.98,17.70,...,11.54,8.79,20.33,6.66,36.58,14.27,5.74,29.99,7.56,22.79
2018-07-04,16.36,9.63,27.94,NaN,20.99,6.00,24.66,13.53,15.19,17.69,...,11.68,8.84,20.25,7.06,36.63,14.23,5.72,30.19,7.52,23.10
2018-07-05,16.63,9.72,27.31,NaN,20.42,5.97,24.09,13.44,15.16,17.63,...,11.63,8.93,20.23,6.95,37.90,14.03,5.58,29.95,7.55,22.88
2018-07-06,16.35,9.80,27.30,NaN,20.79,6.18,23.82,13.54,15.20,17.69,...,11.74,8.89,20.63,6.91,38.39,14.52,5.48,29.04,7.72,23.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-20,17.80,31.97,61.12,NaN,57.00,14.39,44.57,23.10,24.84,30.93,...,14.18,22.38,22.57,8.14,42.91,25.75,11.37,40.49,16.38,46.14
2019-12-23,17.84,32.32,61.03,NaN,57.60,13.73,45.91,23.56,25.00,31.10,...,14.28,22.12,23.23,8.32,42.75,26.54,11.67,40.61,16.35,46.98
2019-12-26,18.09,32.47,61.15,NaN,57.07,13.67,46.43,23.81,25.31,30.95,...,14.46,21.97,23.48,8.32,42.91,26.49,11.80,41.35,16.86,46.60


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2019-07-01,16.40,20.38,30.84,NaN,42.09,11.08,45.48,22.20,25.01,26.20,...,10.39,14.16,17.90,7.74,41.96,21.37,5.12,34.48,10.37,27.18
2019-07-02,16.69,20.85,30.68,NaN,42.41,10.90,44.90,22.18,25.09,26.04,...,10.37,14.65,17.63,7.65,40.20,20.59,5.45,35.41,10.36,27.67
2019-07-03,17.37,20.85,31.13,NaN,44.16,11.15,45.83,22.60,25.43,26.18,...,10.68,14.83,17.67,7.91,40.12,20.94,5.93,35.64,10.43,27.82
2019-07-04,17.49,21.04,33.03,NaN,46.75,11.54,46.58,23.10,25.98,26.43,...,11.05,15.13,17.81,8.25,40.41,21.38,6.24,36.49,10.67,28.70
2019-07-05,17.49,21.20,34.48,NaN,48.45,11.76,46.43,23.28,26.21,27.04,...,11.02,15.89,17.77,8.45,39.39,21.59,6.35,36.18,10.77,29.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-24,12.88,29.38,103.60,NaN,21.10,15.79,28.46,14.74,16.06,25.40,...,13.05,22.10,17.17,6.46,44.35,18.89,14.58,42.15,22.37,34.32
2020-06-25,13.35,29.81,102.61,NaN,21.20,16.50,29.16,15.06,16.44,25.97,...,13.24,22.75,17.00,6.42,44.84,19.42,14.55,42.95,23.91,33.95
2020-06-26,13.13,28.64,100.51,NaN,20.00,16.03,28.11,14.63,15.94,24.94,...,13.08,21.95,16.38,6.25,44.54,18.65,14.29,42.17,24.29,32.33


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2020-01-02,18.07,32.20,63.54,NaN,58.80,13.32,46.96,24.57,26.11,31.22,...,14.63,22.67,23.48,8.46,43.50,27.01,11.73,40.58,17.05,44.83
2020-01-03,17.82,32.20,62.68,NaN,56.76,12.94,46.88,24.18,26.12,30.88,...,14.66,23.16,23.20,8.38,43.18,26.86,11.48,41.05,16.83,43.89
2020-01-06,17.90,31.69,62.29,NaN,55.00,12.81,46.26,24.04,25.65,31.39,...,14.64,22.63,22.70,8.22,42.93,25.51,11.48,40.65,16.87,44.73
2020-01-07,17.94,32.41,62.14,NaN,56.82,13.23,45.91,23.63,25.21,31.40,...,14.80,22.72,23.20,8.30,43.24,25.48,11.65,41.55,17.00,43.90
2020-01-08,17.85,32.77,62.97,NaN,56.92,13.19,45.49,23.27,24.82,31.39,...,14.95,22.42,23.02,8.19,43.25,25.35,11.60,41.97,16.36,43.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-22,14.92,40.08,74.31,NaN,35.27,18.27,34.23,18.74,20.97,26.92,...,13.75,27.41,21.68,12.30,71.27,20.13,15.94,40.14,35.49,31.46
2020-12-23,14.92,40.71,73.95,NaN,37.70,18.19,34.79,19.14,21.45,26.90,...,13.92,27.56,22.01,12.88,71.62,20.19,16.13,40.23,35.40,32.15
2020-12-28,15.17,41.44,75.36,NaN,37.77,18.64,35.18,19.31,21.66,27.45,...,14.04,27.88,22.19,12.73,71.58,20.24,16.59,40.59,36.80,32.32


ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2020-07-01,13.52,28.94,109.21,NaN,20.67,16.85,29.21,14.79,16.30,24.80,...,13.49,23.42,17.46,6.46,43.67,19.16,15.99,41.88,25.56,33.62
2020-07-02,13.34,29.69,105.22,NaN,20.76,16.90,29.29,14.91,16.35,24.65,...,13.57,23.79,17.31,6.47,44.43,19.32,15.39,42.08,26.04,33.41
2020-07-03,13.31,29.00,106.12,NaN,21.15,16.99,29.59,14.91,16.46,24.86,...,14.22,23.86,17.11,6.57,44.25,19.51,15.55,42.77,26.25,33.68
2020-07-06,13.52,29.28,115.28,NaN,22.05,16.99,30.62,15.91,17.46,25.10,...,14.27,24.02,17.22,6.70,45.30,19.76,15.96,42.64,25.86,34.15
2020-07-07,13.60,29.35,114.16,NaN,21.50,17.19,29.40,15.26,16.74,24.89,...,13.92,24.18,16.75,6.56,45.14,19.69,15.86,42.39,26.20,33.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-24,17.35,49.41,68.84,17.28,47.00,15.90,31.31,19.89,23.59,22.91,...,11.57,37.32,18.38,16.92,96.86,27.04,15.77,40.54,33.63,35.23
2021-06-25,16.39,48.98,66.62,16.81,45.97,15.62,30.37,19.38,22.85,22.46,...,11.40,36.64,18.04,16.69,98.06,26.99,15.72,39.82,32.88,33.75
2021-06-28,16.92,50.53,67.94,17.00,45.33,15.90,30.14,19.23,22.74,22.63,...,11.55,37.38,18.23,16.77,96.49,26.68,16.03,39.98,33.33,33.77


In [449]:
df_prices[dates[d]:].index[0]

Timestamp('2021-07-01 00:00:00')

In [446]:
df_prices.loc[ini: fin]

ticker,ABEV3,ALPA4,AMER3,ASAI3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,...,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,VIIA3,VIVT3,WEGE3,YDUQ3
time,,,,,,,,,,,,,,,,,,,,,
2019-07-02,16.69,20.85,30.68,NaN,42.41,10.90,44.90,22.18,25.09,26.04,...,10.37,14.65,17.63,7.65,40.20,20.59,5.45,35.41,10.36,27.67
2019-07-03,17.37,20.85,31.13,NaN,44.16,11.15,45.83,22.60,25.43,26.18,...,10.68,14.83,17.67,7.91,40.12,20.94,5.93,35.64,10.43,27.82
2019-07-04,17.49,21.04,33.03,NaN,46.75,11.54,46.58,23.10,25.98,26.43,...,11.05,15.13,17.81,8.25,40.41,21.38,6.24,36.49,10.67,28.70
2019-07-05,17.49,21.20,34.48,NaN,48.45,11.76,46.43,23.28,26.21,27.04,...,11.02,15.89,17.77,8.45,39.39,21.59,6.35,36.18,10.77,29.00
2019-07-08,17.44,20.75,35.14,NaN,48.73,11.73,46.38,23.43,26.09,26.77,...,10.88,15.78,17.85,8.42,39.85,22.20,6.75,36.74,10.83,30.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-25,16.39,48.98,66.62,16.81,45.97,15.62,30.37,19.38,22.85,22.46,...,11.40,36.64,18.04,16.69,98.06,26.99,15.72,39.82,32.88,33.75
2021-06-28,16.92,50.53,67.94,17.00,45.33,15.90,30.14,19.23,22.74,22.63,...,11.55,37.38,18.23,16.77,96.49,26.68,16.03,39.98,33.33,33.77
2021-06-29,16.72,50.08,67.38,17.05,44.53,15.95,29.94,19.07,22.60,22.29,...,11.33,37.18,17.61,16.97,98.15,25.74,16.02,39.35,34.01,33.14


In [685]:
rates = mt5.copy_rates_range('IBOV', mt5.TIMEFRAME_D1,utc_from, utc_to)

# criamos a partir dos dados obtidos DataFrame
rates_frame = pd.DataFrame(rates)
# convertemos o tempo em segundos no formato datetime
rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
rates_frame['ticker'] = 'IBOV'

In [687]:
rates_frame.to_csv('ibov.csv', index=False)